In [1]:
KB = 1024.0
MB = 1024 * KB
GB = 1024 * MB

In [2]:
QLOG = 2.2 * MB
QCAM = QLOG

RLOG = 56 * MB
DCAM = ECAM = FCAM = 73 * MB

In [3]:
TRAINING = RLOG + DCAM + ECAM + FCAM  # TODO: toggle dcam
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=4.40MB
Dashcam / Training (%) = 1.60%


In [4]:
# What if we increased qcam size and had three of them?
QCAM = QCAM * 4 * 3
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=28.60MB
Dashcam / Training (%) = 10.40%


In [5]:
# comma 3X with OP 0.9.7
RESERVED_BYTES = (4.5 + 3.1 + 1.9) * GB  # 4.5G staging, 3.1G openpilot, 1.9G scons cache
AVAILABLE_BYTES = 88 * GB - RESERVED_BYTES  # 88G partition
# AVAILABLE_BYTES = 100 * GB  # 100 GB

MIN_BYTES = 5 * GB
MIN_PERCENT = 10

def calculate_used_space(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return len(routes_training) * TRAINING + len(routes_dashcam) * DASHCAM

def get_available_percent(routes_training: list[str], routes_dashcam: list[str]) -> float:
  return 100 * (1 - calculate_used_space(routes_training, routes_dashcam) / AVAILABLE_BYTES)

def get_available_bytes(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return AVAILABLE_BYTES - calculate_used_space(routes_training, routes_dashcam)

get_available_percent([], [])

100.0

In [6]:
class Logger:
  route_counter = 0

  def __init__(self):
    self.route_counter = 0

  def __next__(self) -> str:
    log_id = f"{self.route_counter:010}"
    self.route_counter += 1
    return log_id

next(Logger())

'0000000000'

In [7]:
def do_delete_original(routes_training: list[str], routes_dashcam: list[str]):
  # remove earliest route from both lists
  routes_training.pop(0)
  routes_dashcam.pop(0)

do_delete = do_delete_original

In [8]:
# class Deleter:
#   def __init__(self, minimum_bytes: int, minimum_percent: int, )

def deleter(routes_training: list[str], routes_dashcam: list[str]):
  while True:
    out_of_bytes = get_available_bytes(routes_training, routes_dashcam) < MIN_BYTES
    out_of_percent = get_available_percent(routes_training, routes_dashcam) < MIN_PERCENT

    if not (out_of_bytes or out_of_percent):
      return

    # TODO: avoid deleting most recent N preserved segments

    if len(routes_training) == 0 and len(routes_dashcam) == 0:
      return

    do_delete(routes_training, routes_dashcam)

In [9]:
import pandas as pd


def loop(count: int):
  logger = Logger()
  routes_training = []
  routes_dashcam = []

  header = ["training_count", "dashcam_count", "training_bytes", "dashcam_bytes", "total_bytes", "available_bytes", "available_percent"]
  rows = []

  for _ in range(count):
    # 1. log new segment
    log_id = next(logger)
    routes_training.append(log_id)
    routes_dashcam.append(log_id)

    # 2. run deleter
    deleter(routes_training, routes_dashcam)

    # 3. append stats
    training_count, dashcam_count = len(routes_training), len(routes_dashcam)
    rows.append([
      training_count,
      dashcam_count,
      training_count * TRAINING,
      dashcam_count * DASHCAM,
      training_count * TRAINING + dashcam_count * DASHCAM,
      get_available_bytes(routes_training, routes_dashcam),
      get_available_percent(routes_training, routes_dashcam),
    ])

  return pd.DataFrame(rows, columns=header)


df = loop(1024)
df

,training_count,dashcam_count,training_bytes,dashcam_bytes,total_bytes,available_bytes,available_percent
0,1,1,2.883584e+08,2.998927e+07,3.183477e+08,8.397039e+10,99.622313
1,2,2,5.767168e+08,5.997855e+07,6.366953e+08,8.365204e+10,99.244626
2,3,3,8.650752e+08,8.996782e+07,9.550430e+08,8.333369e+10,98.866939
3,4,4,1.153434e+09,1.199571e+08,1.273391e+09,8.301534e+10,98.489252
4,5,5,1.441792e+09,1.499464e+08,1.591738e+09,8.269699e+10,98.111564
...,...,...,...,...,...,...,...
1019,238,238,6.862930e+10,7.137447e+09,7.576675e+10,8.521987e+09,10.110470
1020,238,238,6.862930e+10,7.137447e+09,7.576675e+10,8.521987e+09,10.110470
1021,238,238,6.862930e+10,7.137447e+09,7.576675e+10,8.521987e+09,10.110470
1022,238,238,6.862930e+10,7.137447e+09,7.576675e+10,8.521987e+09,10.110470


In [10]:
f"{df["dashcam_count"].iloc[-1]} minutes"

'238 minutes'